# House Prices Competition

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(rc={'figure.figsize':(22,8.27)})

In [ ]:
train = pd.read_csv('./data/train.csv', index_col='Id')
test = pd.read_csv('./data/test.csv', index_col='Id')

data = pd.concat([train, test])

In [ ]:
data.head()

In [ ]:
data.isnull().sum(axis=0).head(10)

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
for col in data.columns:
    if data[col].isna().sum() > 0.7 * data.shape[0]:
        data.drop(col, axis=1, inplace=True)

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
data.info()

In [ ]:
na_columns = data.drop('SalePrice', axis=1).columns[(data.drop('SalePrice', axis=1).isna().sum() > 0)]
data[na_columns[1]].dtype == 'int'

In [ ]:
for col in na_columns:
    if str(data[col].dtype) == 'float64':
        data[col].fillna(data[col].median(), inplace=True)
    else:
        data[col].fillna(data[col].mode(), inplace=True)
        data[col] = data[col].replace({None: data[col].mode()[0]})

In [ ]:
data['FireplaceQu'].replace({None: data['FireplaceQu'].mode()[0]})

In [ ]:
data['FireplaceQu'].mode()

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
data['FireplaceQu'].isnull().sum()

In [ ]:
'FireplaceQu' in na_columns